# Scenarios
In this notebook we'll show how to run different scenario's on your portfolio, to see what a certain action would do to your temperature score.

### Setting up
We'll set up the imports, data providers, and load the portfolio. For more details about this, please have a look at notebook one (getting started).

In [1]:
# Only required if you didn't install the module yet
!pip install SBTi

Looking in indexes: http://rdnexus1.ortec.finance:8081/nexus/repository/pypi-all/simple


In [2]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [3]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/example_portfolio.csv", "example_portfolio.csv")

In [4]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")
companies = [SBTi.interfaces.PortfolioCompany.parse_obj(company) for company in portfolio.to_dict(orient="records")]
portfolio_data = SBTi.utils.get_data([provider], companies)

## Base scenario
To determine what the effect is of running a certain scenario, we first need a base line. To do so, we run the calculate method of the temperature score, without defining a scenario. We'll determine the effect of our scenario's by looking at the aggregated temperature score for the short timeframe in the combined s1s2s3 scope. Of course you can use multiple, or different criteria.

In [14]:
time_frames = list(SBTi.interfaces.ETimeFrames) 
scopes = [EScope.S1S2, EScope.S3, EScope.S1S2S3]

In [30]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes)
base_scenario = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
base_scenario_aggregated['short']['S1S2S3']["all"]["score"]

2.670272727272727

## Scenario 1
In scenario 1 we engage companies to set targets. This means that the default score (i.e. the score when a company didn't set a target) will be set to 2 degrees celcius.

In [31]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [33]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, scenario=scenario, aggregation_method=PortfolioAggregationMethod.WATS)
scenario_one = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
scenario_one["short"]["S1S2S3"]["all"]["score"]

KeyError: 'short'

## Scenario 2
In scenario 2 we engage companies to have their targets validated by the SBTi. This means that the maximum score for all targets will be capped at 1.75 degrees celcius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.APPROVED_TARGETS
scenario.engagement_type = None
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, aggregation_method=PortfolioAggregationMethod.WATS)
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
scenario_two["short"]["s1s2s3"]["all"]["score"]

## Scenario 3a
In scenario 3a we engage the top 10 contributors to set targets. This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees celcius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, aggregation_method=PortfolioAggregationMethod.WATS)
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
scenario_two["short"]["s1s2s3"]["all"]["score"]

## Scenario 3b
In scenario 3b we engage the top 10 contributors to have their targets validated by the SBTi. This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees celcius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, aggregation_method=PortfolioAggregationMethod.WATS)
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
scenario_two["short"]["s1s2s3"]["all"]["score"]

## Scenario 4a
In scenario 4a we engage the predefined companoies to set targets. This means that their default score (i.e. the score when a company didn't set a target) will be set to 2 degrees celcius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, aggregation_method=PortfolioAggregationMethod.WATS)
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
scenario_two["short"]["s1s2s3"]["all"]["score"]

## Scenario 4b
In scenario 4b we engage the predefined companies to have their targets validated by the SBTi. This means that the maximum score for all the targets of these companies will be capped at 1.75 degrees celcius.

In [ ]:
scenario = Scenario()
scenario.scenario_type = ScenarioType.HIGHEST_CONTRIBUTORS_APPROVED
scenario.engagement_type = EngagementType.SET_SBTI_TARGETS
scenario.aggregation_method = PortfolioAggregationMethod.WATS
scenario.grouping = None

In [ ]:
temperature_score = TemperatureScore(time_frames=time_frames, scopes=scopes, aggregation_method=PortfolioAggregationMethod.WATS)
temperature_score.set_scenario(scenario)
scenario_two = temperature_score.calculate(portfolio_data.copy())
base_scenario_aggregated = temperature_score.aggregate_scores(base_scenario)
scenario_two["short"]["s1s2s3"]["all"]["score"]